# Выбор локации для скважины

Допустим, вы работаете в добывающей компании «ГлавРосГосНефть». Нужно решить, где бурить новую скважину.

Вам предоставлены пробы нефти в трёх регионах: в каждом 10 000 месторождений, где измерили качество нефти и объём её запасов. Постройте модель машинного обучения, которая поможет определить регион, где добыча принесёт наибольшую прибыль. Проанализируйте возможную прибыль и риски техникой *Bootstrap.*

Шаги для выбора локации:

- В избранном регионе ищут месторождения, для каждого определяют значения признаков;
- Строят модель и оценивают объём запасов;
- Выбирают месторождения с самым высокими оценками значений. Количество месторождений зависит от бюджета компании и стоимости разработки одной скважины;
- Прибыль равна суммарной прибыли отобранных месторождений.

## Загрузка и подготовка данных

Импортируем необходимые библиотеки

In [1]:
import pandas as pd
import numpy as np
from scipy import stats as st
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from sklearn.linear_model import LinearRegression

Загрузим датафреймы для 3 регионов и выведем общую информацию 

In [2]:
try:   
    data_0 = pd.read_csv('/datasets/geo_data_0.csv')
    data_1 = pd.read_csv('/datasets/geo_data_1.csv')
    data_2 = pd.read_csv('/datasets/geo_data_2.csv')
except:
    data_0 = pd.read_csv('https://code.s3.yandex.net/datasets/geo_data_0.csv')
    data_1 = pd.read_csv('https://code.s3.yandex.net/datasets/geo_data_1.csv')
    data_2 = pd.read_csv('https://code.s3.yandex.net/datasets/geo_data_2.csv')

data_0.info(), data_1.info(), data_2.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100000 entries, 0 to 99999
Data columns (total 5 columns):
 #   Column   Non-Null Count   Dtype  
---  ------   --------------   -----  
 0   id       100000 non-null  object 
 1   f0       100000 non-null  float64
 2   f1       100000 non-null  float64
 3   f2       100000 non-null  float64
 4   product  100000 non-null  float64
dtypes: float64(4), object(1)
memory usage: 3.8+ MB
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100000 entries, 0 to 99999
Data columns (total 5 columns):
 #   Column   Non-Null Count   Dtype  
---  ------   --------------   -----  
 0   id       100000 non-null  object 
 1   f0       100000 non-null  float64
 2   f1       100000 non-null  float64
 3   f2       100000 non-null  float64
 4   product  100000 non-null  float64
dtypes: float64(4), object(1)
memory usage: 3.8+ MB
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100000 entries, 0 to 99999
Data columns (total 5 columns):
 #   Column   Non-Null 

(None, None, None)

Выведем первые 5 строк датафрейма для первого региона  

In [3]:
data_0.head()

,id,f0,f1,f2,product
0,txEyH,0.705745,-0.497823,1.221170,105.280062
1,2acmU,1.334711,-0.340164,4.365080,73.037750
2,409Wp,1.022732,0.151990,1.419926,85.265647
3,iJLyR,-0.032172,0.139033,2.978566,168.620776
4,Xdl7t,1.988431,0.155413,4.751769,154.036647


Выведем первые 5 строк датафрейма для второго региона  

In [4]:
data_1.head()

,id,f0,f1,f2,product
0,kBEdx,-15.001348,-8.276000,-0.005876,3.179103
1,62mP7,14.272088,-3.475083,0.999183,26.953261
2,vyE1P,6.263187,-5.948386,5.001160,134.766305
3,KcrkZ,-13.081196,-11.506057,4.999415,137.945408
4,AHL4O,12.702195,-8.147433,5.004363,134.766305


Выведем первые 5 строк датафрейма для третьего региона  

In [5]:
data_2.head()

,id,f0,f1,f2,product
0,fwXo0,-1.146987,0.963328,-0.828965,27.758673
1,WJtFt,0.262778,0.269839,-2.530187,56.069697
2,ovLUW,0.194587,0.289035,-5.586433,62.871910
3,q6cA6,2.236060,-0.553760,0.930038,114.572842
4,WPMUX,-0.515993,1.716266,5.899011,149.600746


Все 3 датафрейма имеют одинаковую структуру. Заметим, что все признаки в датафрейме важны для обучения модели, кроме признака “id”. Его мы уберем при разделении на выборки features и target. 

## Обучение и проверка модели

Создадим функцию для разделения датафреймов регионов добычи на выборки  features и target

In [6]:
def split_features_target(data):
    features = data.drop(['product', 'id'], axis=1)
    target = data['product']
    return features, target

Применим написанную нами функцию к датафремам 

In [7]:
features_0, target_0 = split_features_target(data_0)
features_1, target_1 = split_features_target(data_1)
features_2, target_2 = split_features_target(data_2)

Разделим наши выборки для первого региона на тестовую и валидационную выборку, а также проверим их размер 

In [8]:
features_train_0, features_valid_0, target_train_0, target_valid_0 = train_test_split(
    features_0, target_0, test_size=0.25, random_state=12345)

features_train_0.shape[0], features_valid_0.shape[0], target_train_0.shape[0], target_valid_0.shape[0]

(75000, 25000, 75000, 25000)

Проделаем тоже самое для второго и третьего региона  

In [9]:
features_train_1, features_valid_1, target_train_1, target_valid_1 = train_test_split(
    features_1, target_1, test_size=0.25, random_state=12345)

features_train_1.shape[0], features_valid_1.shape[0], target_train_1.shape[0], target_valid_1.shape[0]

(75000, 25000, 75000, 25000)

In [10]:
features_train_2, features_valid_2, target_train_2, target_valid_2 = train_test_split(
    features_2, target_2, test_size=0.25, random_state=12345)

features_train_2.shape[0], features_valid_2.shape[0], target_train_2.shape[0], target_valid_2.shape[0]

(75000, 25000, 75000, 25000)

Обучим модель линейной регрессии на валидационной выборке для первого региона, посчитаем средний запас предсказанного сырья а также выведем значение RMSE

In [11]:
model_0 = LinearRegression()
model_0.fit(features_train_0, target_train_0)
predict_0 = model_0.predict(features_valid_0)
rmse_0 = mean_squared_error(target_valid_0, predict_0) ** .5
mean_predict_0 = predict_0.mean()
print(f'Средний запас предсказанного сырья: {mean_predict_0}, \
RMSE модели линейной регрессии на валидационной выборке: {rmse_0}')

Средний запас предсказанного сырья: 92.59256778438035, RMSE модели линейной регрессии на валидационной выборке: 37.5794217150813


Проделаем тоже самое для второго и третьего региона  

In [12]:
model_1 = LinearRegression()
model_1.fit(features_train_1, target_train_1)
predict_1 = model_1.predict(features_valid_1)
rmse_1 = mean_squared_error(target_valid_1, predict_1) ** .5
mean_predict_1 = predict_1.mean()
print(f'Средний запас предсказанного сырья: {mean_predict_1}, \
RMSE модели линейной регрессии на валидационной выборке: {rmse_1}')

Средний запас предсказанного сырья: 68.728546895446, RMSE модели линейной регрессии на валидационной выборке: 0.893099286775617


In [13]:
model_2 = LinearRegression()
model_2.fit(features_train_2, target_train_2)
predict_2 = model_2.predict(features_valid_2)
rmse_2 = mean_squared_error(target_valid_2, predict_2) ** .5
mean_predict_2 = predict_2.mean()
print(f'Средний запас предсказанного сырья: {mean_predict_2}, \
RMSE модели линейной регрессии на валидационной выборке: {rmse_2}')

Средний запас предсказанного сырья: 94.96504596800489, RMSE модели линейной регрессии на валидационной выборке: 40.02970873393434


Средний запас предсказанного сырья меньше всего оказался в регионе № 2, но показатель RMSE равен 0,89, что означает что модель предсказывает достаточно точно в данном регионе. Самый большой средний запас в регионе № 3, но показатель RMSE равен 40, что почти в 2 раза меньше предсказанного запаса! Для определения наиболее выгодного региона для добычи необходимо дальнейшее исследование. 

## Подготовка к расчёту прибыли

Посчитаем необходимый объем добычи для одной скважины. Учтем что бюджет на разработку равен 10 млрд., цена за 1 тыс. баррелей равна 450 тыс., количество скважин для разработки – 200.  

In [14]:
BUDGET = 10_000_000_000
PRICE_PER_UNIT = 450000
NUMBER_OF_WELLS = 200

In [15]:
min_quantity = BUDGET / PRICE_PER_UNIT / NUMBER_OF_WELLS
min_quantity

111.11111111111111

Получилось, что для того, чтобы “уйти в ноль” скважина должна вырабатывать 111.11111111111111 тыс. баррелей   

Напишим функцию для расчёта прибыли по выбранным скважинам и предсказаниям модели

In [16]:
def sum_profit(predict, target):
    predict_top_200 = predict.sort_values(ascending=False)
    sum_profit_wells = sum(target[predict_top_200.index][:200]) * PRICE_PER_UNIT
    return sum_profit_wells - BUDGET

Посчитаем прибыль для 200 самых крупных по объему выработки скважин для 3 регионов    

In [17]:
predict_0 = pd.Series(predict_0, index = target_valid_0.index)
predict_1 = pd.Series(predict_1, index = target_valid_1.index)
predict_2 = pd.Series(predict_2, index = target_valid_2.index)
sum_profit(predict_0, target_valid_0), sum_profit(predict_1, target_valid_1),sum_profit(predict_2, target_valid_2)

(3320826043.1398544, 2415086696.681551, 2710349963.5998363)

## Расчёт прибыли и рисков 

Напишем функцию для подсчета риска убытка

In [18]:
def procent_loss(row):
    return (row < 0).mean() * 100

Применим технику Bootstrap с 1000 выборок, чтобы найти распределение прибыли для первого региона

In [19]:
state = np.random.RandomState(12345)
    
values_0 = []
for i in range(1000):
    predict_subsample_0 = predict_0.sample(n=500, replace=True, random_state=state)
    values_0.append(sum_profit(predict_subsample_0, target_valid_0))

values_0 = pd.Series(values_0)

mean_0 = values_0.mean()
lower_0 = values_0.quantile(0.025)
upper_0 = values_0.quantile(0.975)
procent_loss_0 = procent_loss(values_0)

print(f'Средняя выручка: {mean_0}руб., 95%-й доверительный интервал: {lower_0, upper_0}, риск убытка {procent_loss_0:.1f}%' )


Средняя выручка: 396164984.80237156руб., 95%-й доверительный интервал: (-111215545.89049521, 909766941.5534207), риск убытка 6.9%


Применим технику Bootstrap с 1000 выборок, чтобы найти распределение прибыли для второго региона

In [20]:
values_1 = []
for i in range(1000):
    predict_subsample_1 = predict_1.sample(n=500, replace=True, random_state=state)
    values_1.append(sum_profit(predict_subsample_1,target_valid_1))

values_1 = pd.Series(values_1)

mean_1 = values_1.mean()
lower_1 = values_1.quantile(0.025)
upper_1 = values_1.quantile(0.975)
procent_loss_1 = procent_loss(values_1)

print(f'Средняя выручка: {mean_1}руб., 95%-й доверительный интервал: {lower_1, upper_1}, риск убытка {procent_loss_1:.1f}%' )

Средняя выручка: 461155817.27726144руб., 95%-й доверительный интервал: (78050810.75175706, 862952060.2637427), риск убытка 0.7%


Применим технику Bootstrap с 1000 выборок, чтобы найти распределение прибыли для третьего региона

In [21]:
values_2 = []
for i in range(1000):
    predict_subsample_2 = predict_2.sample(n=500, replace=True, random_state=state)
    values_2.append(sum_profit(predict_subsample_2,target_valid_2))

values_2 = pd.Series(values_2)

mean_2 = values_2.mean()
lower_2 = values_2.quantile(0.025)
upper_2 = values_2.quantile(0.975)
procent_loss_2 = procent_loss(values_2)

print(f'Средняя выручка: {mean_2}руб., 95%-й доверительный интервал: {lower_2, upper_2}, риск убытка {procent_loss_2:.1f}%' )

Средняя выручка: 392950475.170605руб., 95%-й доверительный интервал: (-112227625.37857184, 934562914.5511599), риск убытка 6.5%


После проведенных исследований для разработки рекомендую регион №2, т.к. у него наименьший риск убытка, наибольший 2.5%-ый квантиль и наибольшая средняя выручка. У других регионов риск убытка в разы больше и 2.5%-ый квантиль имеет отрицательное значение.

## Вывод

В первом разделе “Загрузка и подготовка данных” были загружены датафреймы для 3 различных регионов разработки месторождений. Полученная общая информация о них, и было выявлено что 3 датафрейма имеют одинаковую структуру. 

Во втором разделе “Обучение и проверка модели” датасеты были разделены на две выборки для обучения, удален столбец с уникальным идентификатором скважины, так как он не несет никакой пользы для обучения. Впоследствии выборки были дополнительно разделены на обучающую и валидационную выборку. Для обучения подошла только модель линейной регрессии, остальные модели недостаточно предсказуемые. Посчитан средний запас предсказанного сырья и метрика RMSE. Полученные данные показали, что необходимо дальнейшее исследование.

В третьем разделе “Подготовка к расчету прибыли” был рассчитан минимальный объем произведенной продукции для одной скважины, его значение 111.11 тыс. баррелей. Также была написана функция для расчета прибыли по выбранным скважинам, необходимая для исследование в четвертом разделе. 

В четвертом разделе “Расчет прибыли и рисков” применили технику Bootstrap и посчитали среднюю выручку, доверительный интервал и риск убытка для каждого региона. По этим данным была сформулирована рекомендация для наилучшего региона для разработки. Им оказался регион №2, так как риск убытка в этим регионе показал минимальное значение,средняя выручка максимальна среди всех регионов, а доверительный 95-и % интервал имеет значение (78050810, 862952060) и его правая граница (2.5% квантиль) имеет самое высокое значение среди других регионов. 
